In [48]:
import json
from graphviz import Digraph
# importOS
import os

class State:
    def __init__(self, label=None):
        self.state_id = None
        self.is_terminating = False
        self.next_states = {}
        

def disjunction(initial_state, r_index, regex, states, last_s_index):
    n_begin_s_index = last_s_index + 1
    n_last_s_index = n_begin_s_index
    n_prev_state = n_begin_s_index
    # r_index from element after '|'
    while r_index < len(regex):
        if regex[r_index] == '(':
            print("")
        elif regex[r_index] == '|':
            print("or")
            return r_index,n_last_s_index,0
            # call or function
        elif regex[r_index] == '*':
            # print("")
            new_state = State()
            new_state.state_id = n_begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : n_begin_s_index +1} )
            new_state.next_states.update( {'e2' : n_prev_state} )
            states[n_begin_s_index] = new_state
            states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
            n_begin_s_index =n_begin_s_index+1
            r_index +=1
            n_last_s_index = n_begin_s_index
            
            
            
        elif regex[r_index] == '+':
            new_state = State()
            new_state.state_id = n_begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : n_begin_s_index +1} )
            new_state.next_states.update( {'e2' : n_prev_state} )
            states[n_begin_s_index] = new_state
            # states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
            n_begin_s_index =n_begin_s_index+1
            r_index +=1
            n_last_s_index = n_begin_s_index
        elif regex[r_index] == '?':
            # print("")
            new_state = State()
            new_state.state_id = n_begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : n_begin_s_index +1} )
            # new_state.next_states.update( {'e2' : n_prev_state} )
            states[n_begin_s_index] = new_state
            states[n_prev_state].next_states.update( {'e' : n_begin_s_index +1} )
            n_begin_s_index =n_begin_s_index+1
            r_index +=1
            n_last_s_index = n_begin_s_index
        else:
            new_state = State()
            new_state.state_id = n_begin_s_index
            new_state.is_terminating = False
            # 
            new_state.next_states.update( {regex[r_index] : n_begin_s_index +1} )
            states[n_begin_s_index] = new_state
            n_prev_state = n_begin_s_index
            n_begin_s_index =n_begin_s_index+1
            r_index +=1
            n_last_s_index = n_begin_s_index
            
    finish =1
    return -1,n_last_s_index,finish

def save_json(states,initial_state):
    state_data = {
    "startingState": "S" + str(initial_state),
    "states": {}
    }

    for state_id, state_obj in states.items():
        state_info = {
            "state_id": state_obj.state_id,
            "isTerminatingState": state_obj.is_terminating,
            "Transitions": {}
        }

        if not state_obj.is_terminating:
            for transition, next_state_id in state_obj.next_states.items():
                state_info["Transitions"][transition] = "S" + str(next_state_id)
        # else:
            # remove the transitions key if the state is terminating
            # state_info.pop("Transitions")

        state_data["states"]["S" + str(state_obj.state_id)] = state_info

    # Writing to JSON file
    with open("states.json", "w") as json_file:
        json.dump(state_data, json_file, indent=4)

def thompsons_construction(regex):
    r_index = 0
    states = {}
    s_0 = State()
    s_0.state_id =0
    s_0.is_terminating =False
    states.update({0:s_0})
    # point to the first state
    begin_s_index = 0
    # point to the last state
    last_s_index = 0
    # point to the initial state (starting state)
    initial_state = 0
    # previous element that will used in case of repetition
    prev_state = 0
    while r_index < len(regex):
        if regex[r_index] == '(':
            print("")
        elif regex[r_index] == '|':
            print("or")
            # call or function
            begin_disjunction = last_s_index + 1
            r_index,d_last_s_index, finish = disjunction(initial_state, r_index+1, regex, states, last_s_index)
            # new initial state
            new_state = State()
            new_state.state_id = d_last_s_index + 1
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : begin_disjunction} )
            new_state.next_states.update( {'e2' : initial_state  } )
            states[d_last_s_index + 1] = new_state
            #new end state
            new_state = State()
            new_state.state_id = d_last_s_index + 2
            new_state.is_terminating = False
            states[d_last_s_index + 2] = new_state
            
            # new 
            new_state = State()
            new_state.state_id = last_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e' : d_last_s_index + 2} )
            states[last_s_index] = new_state

            # new
            # states[d_last_s_index].next_states.update( {'e' : d_last_s_index + 2} )
            new_state = State()
            new_state.state_id = d_last_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e' : d_last_s_index + 2} )
            states[d_last_s_index] = new_state
            
            last_s_index = d_last_s_index + 2
            initial_state = d_last_s_index + 1
            prev_state = d_last_s_index + 1
            begin_s_index = d_last_s_index + 2
            
            
            
            if finish == 1:
                r_index = len(regex)
        elif regex[r_index] == '*':
            # print("")
            # zero or more 
            new_state = State()
            new_state.state_id = begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : begin_s_index +1} )
            # in case of more will go back to the prev state
            new_state.next_states.update( {'e2' : prev_state} )
            states[begin_s_index] = new_state
            # for zero will go to next state with e moves
            states[prev_state].next_states.update( {'e' : begin_s_index +1} )
            begin_s_index =begin_s_index+1
            r_index +=1
            last_s_index = begin_s_index
        
        elif regex[r_index] == '+':
            # print("")
            # print("")
            # zero or more 
            new_state = State()
            new_state.state_id = begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : begin_s_index +1} )
            # in case of more will go back to the prev state
            new_state.next_states.update( {'e2' : prev_state} )
            states[begin_s_index] = new_state
            # for zero will go to next state with e moves
            # states[prev_state].next_states.update( {'e' : begin_s_index +1} )
            begin_s_index =begin_s_index+1
            r_index +=1
            last_s_index = begin_s_index
            
        elif regex[r_index] == '?':
            # print("")
            # print("")
            # zero or more 
            new_state = State()
            new_state.state_id = begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {'e1' : begin_s_index +1} )
            # in case of more will go back to the prev state
            # new_state.next_states.update( {'e2' : prev_state} )
            states[begin_s_index] = new_state
            # for zero will go to next state with e moves
            states[prev_state].next_states.update( {'e' : begin_s_index +1} )
            begin_s_index =begin_s_index+1
            r_index +=1
            last_s_index = begin_s_index
        else:
            new_state = State()
            new_state.state_id = begin_s_index
            new_state.is_terminating = False
            new_state.next_states.update( {regex[r_index] : begin_s_index +1} )
            states[begin_s_index] = new_state
            prev_state = begin_s_index
            begin_s_index =begin_s_index+1
            r_index +=1
            last_s_index = begin_s_index
    # print loop and print the states
    # print each state and its transitions like json
    
    new_state = State()
    new_state.state_id = last_s_index
    new_state.is_terminating = True
    states[last_s_index] = new_state

    '''
    { "startingState": "S0", "S0": { "isTerminatingState": false, "A": "S1", "B": "S0" }, "S1": { "isTerminatingState": true, "A": "S1", "B": "S1" } }
    
    '''
    print("{ \"startingState\": \"S"+str(initial_state)+"\",")
    #  sort states according to their state_id
    
    states = dict(sorted(states.items()))
    for state in states:
        print("  S"+str(states[state].state_id) + ": {")
        print("isTerminatingState: " + str(states[state].is_terminating) + ",")
        if states[state].is_terminating:
            print("}")
        else:
            print("Transitions: {")
            for transition in states[state].next_states:
                print(transition + ": " + "S" + str(states[state].next_states[transition]) + ",")
            print("}")
            print("}")
    print("}")
    
    
    save_json(states,initial_state)
    
    
    return 0

def draw_graph():
    # "C:\Program Files\Graphviz\bin\dot.exe" run this command in the terminal to install graphviz
    # ! "C:\Program Files\Graphviz\bin\dot.exe"
    os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
    # Load state machine definition from JSON file
    with open("states.json", "r") as json_file:
        state_machine_definition = json.load(json_file)
    #  want to set direction to left to right


    # Initialize the Digraph object
    dot = Digraph()

    # Set the direction of the graph
    dot.attr(rankdir="LR")

    # Add states to the graph
    for state_id, state_info in state_machine_definition["states"].items():
        label = f"{state_id}"
        if state_info['isTerminatingState']:
            dot.node(state_id, label=label, shape="doublecircle")
        else:
            dot.node(state_id, label=label)

    # Add transitions to the graph
    for state_id, state_info in state_machine_definition["states"].items():
        for event, next_state_id in state_info["Transitions"].items():
            dot.edge(state_id, next_state_id, label=event)

    # Save the graph to a file
    dot.render("state_machine_graph", format="png", cleanup=True)


def main():
    regex = input("Enter a regular expression: ")
    nfa = thompsons_construction(regex)
    draw_graph()


if __name__ == "__main__":
    main()


or
or
or
or
or
{ "startingState": "S12",
  S0: {
isTerminatingState: False,
Transitions: {
a: S1,
}
}
  S1: {
isTerminatingState: False,
Transitions: {
e: S5,
}
}
  S2: {
isTerminatingState: False,
Transitions: {
b: S3,
}
}
  S3: {
isTerminatingState: False,
Transitions: {
e: S5,
}
}
  S4: {
isTerminatingState: False,
Transitions: {
e1: S2,
e2: S0,
}
}
  S5: {
isTerminatingState: False,
Transitions: {
e: S9,
}
}
  S6: {
isTerminatingState: False,
Transitions: {
c: S7,
}
}
  S7: {
isTerminatingState: False,
Transitions: {
e: S9,
}
}
  S8: {
isTerminatingState: False,
Transitions: {
e1: S6,
e2: S4,
}
}
  S9: {
isTerminatingState: False,
Transitions: {
e: S13,
}
}
  S10: {
isTerminatingState: False,
Transitions: {
d: S11,
}
}
  S11: {
isTerminatingState: False,
Transitions: {
e: S13,
}
}
  S12: {
isTerminatingState: False,
Transitions: {
e1: S10,
e2: S8,
}
}
  S13: {
isTerminatingState: True,
}
}


In [45]:
import json
from graphviz import Digraph
# importOS
import os

# "C:\Program Files\Graphviz\bin\dot.exe" run this command in the terminal to install graphviz
# ! "C:\Program Files\Graphviz\bin\dot.exe"
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
# Load state machine definition from JSON file
with open("states.json", "r") as json_file:
    state_machine_definition = json.load(json_file)
#  want to set direction to left to right


# Initialize the Digraph object
dot = Digraph()

# Set the direction of the graph
dot.attr(rankdir="LR")

# Add states to the graph
for state_id, state_info in state_machine_definition["states"].items():
    label = f"{state_id}"
    if state_info['isTerminatingState']:
        dot.node(state_id, label=label, shape="doublecircle")
    else:
        dot.node(state_id, label=label)

# Add transitions to the graph
for state_id, state_info in state_machine_definition["states"].items():
    for event, next_state_id in state_info["Transitions"].items():
        dot.edge(state_id, next_state_id, label=event)

# Save the graph to a file
dot.render("state_machine_graph", format="png", cleanup=True)


'state_machine_graph.png'